# HaploADMIXTURE.jl

This software package is an open-source Julia implementation of HaploADMIXTURE, ancestry inference by modeling haplotypes. By modeling haplotypes, we use information between nearby SNPs, obtaining more accurate ancestry estimates.

It supports acceleartion through multithreading and graphic processing units (GPUs). By directly utilizing the data format of the PLINK BED file, the memory usage is highly efficient. 

It estimates ancestry with maximum-likelihood method for a large SNP genotype datasets, where individuals are assumed to be unrelated. The input is binary PLINK 1 BED-formatted file (`.bed`). Also, you will need an idea of $K$, the number of ancestral populations. One possible way to figure out a good value of $K$ is through Akaike information criterion. If the number of SNPs is too large, you may choose to run on a subset of SNPs selected by their information content, using the blockwise [sparse $K$-means via feature ranking](https://github.com/kose-y/SKFR.jl) (SKFR) method. When SKFR is applied, it selects given number of blocks of two nearby SNPs.

## Installation

This package requires Julia v1.7 or later, which can be obtained from
<https://julialang.org/downloads/> or by building Julia from the sources in the
<https://github.com/JuliaLang/julia> repository.

The package can be installed by running the following code:
```julia
using Pkg
pkg"add https://github.com/kose-y/SKFR.jl"
pkg"add https://github.com/OpenMendel/OpenADMIXTURE.jl"
pkg"add https://github.com/OpenMendel/HaploADMIXTURE.jl"
```
For running the examples below, the following are also necessary. 
```julia
pkg"add SnpArrays DelimitedFiles StableRNGs"
```

For GPU support, an Nvidia GPU is required. Also, the following package has to be installed:
```julia
pkg"add CUDA"
```

## Basic Usage

We first import necessary packages:

In [1]:
using LinearAlgebra, Random, SnpArrays, StableRNGs
using HaploADMIXTURE
using DelimitedFiles

We will use the PLINK file included in the `SnpArrays` package, whose path is obtained by:

In [2]:
filename = SnpArrays.datadir("EUR_subset.bed");

This file contains information on 54051 single nucleotide polymorphisms (SNP)s of 379 samples. The main driver function for admixture proportion estimation is called `run_admixture()`. 

In [ ]:
d, clusters, aims = HaploADMIXTURE.run_admixture(filename, 379, 27025, 4; T=Float64, use_gpu=false, rng=StableRNG(7856), admix_rtol=1e-5)

Using /home/kose/.julia/packages/SnpArrays/lx5Kb/src/../data/EUR_subset.bed as input.
Loading genotype data...
Loaded 379 samples and 27025 SNPs


[ Info: EM iter 1, ll: -1.4510327066059288e7
[ Info: EM iter 2, ll: -1.381644526909224e7
[ Info: EM iter 3, ll: -1.3630461295722805e7
[ Info: EM iter 4, ll: -1.3583905487448066e7
[ Info: EM iter 5, ll: -1.3560929046308124e7


initial ll: -1.3560929046308124e7
  0.850404 seconds (4 allocations: 64 bytes)
  0.000264 seconds (27 allocations: 12.156 KiB)
  0.998668 seconds (4 allocations: 64 bytes)
  0.134181 seconds (27 allocations: 12.344 KiB)
  0.849196 seconds (4 allocations: 64 bytes)
  0.000252 seconds (27 allocations: 12.156 KiB)
  0.997638 seconds (4 allocations: 64 bytes)
  0.132407 seconds (27 allocations: 12.344 KiB)


[ Info: Iteration 1: ll=-1.3442143002068464e7, reldiff = 0.008759432619551937, ll_basic=-1.3439421296893205e7, ll_qn=-1.3442143002068464e7


  4.927956 seconds (123.29 k allocations: 21.585 MiB, 0.53% compilation time)


  0.851355 seconds (4 allocations: 64 bytes)
  0.000242 seconds (27 allocations: 12.156 KiB)
  0.998224 seconds (4 allocations: 64 bytes)
  0.122307 seconds (27 allocations: 12.344 KiB)
  0.851050 seconds (4 allocations: 64 bytes)
  0.000246 seconds (27 allocations: 12.156 KiB)
  0.998467 seconds (4 allocations: 64 bytes)
  0.121218 seconds (27 allocations: 12.344 KiB)


[ Info: Iteration 2: ll=-1.3383453322742362e7, reldiff = 0.004366095444533735, ll_basic=-1.3388259907460274e7, ll_qn=-1.3383453322742362e7


  4.859687 seconds (116.22 k allocations: 24.416 MiB)


  0.848979 seconds (4 allocations: 64 bytes)
  0.000273 seconds (27 allocations: 12.156 KiB)
  0.998561 seconds (4 allocations: 64 bytes)
  0.112486 seconds (27 allocations: 12.344 KiB)
  0.848001 seconds (4 allocations: 64 bytes)
  0.000256 seconds (27 allocations: 12.156 KiB)
  0.998448 seconds (4 allocations: 64 bytes)
  0.110201 seconds (27 allocations: 12.344 KiB)


[ Info: Iteration 3: ll=-1.3363811617880553e7, reldiff = 0.0014676111156177924, ll_basic=-1.3367645714833554e7, ll_qn=-1.3363811617880553e7


The first argument is the path to the PLINK 1 `.bed`, and the second argument is the number of populations. The second through fourth arguments are: 
- `I`: Number of individuals. We use first `I` individuals in the PLINK file for the analysis.
- `J`: Number of pairs of SNPs to be used for analysis. We use the first `2J` SNPs in the PLINK file for the analysis.
- `K`: Number of populations.

After the semicolon are the keyword arguments:
- `T`: Precision of the estimation. `Float64` or `Float32`. Default `Float64`. 
- `use_gpu`: Whether to use GPU for estimation. Default `false`. 
- `rng`: Random number generator. Default `Random.GLOBAL_RNG`. 
- `prefix`: Prefix of the PLINK file only with the SNPs selected using SKFR. The output file is named `$(prefix)_$(K)_$(sparsity)aims.bed`. 
- `sparsity`: Number of pairs of SNPs selected by SKFR. Default `nothing` and skip SKFR. 
- `skfr_tries`: Runs SKFR this many times and choose the best clustering. Default 1. 
- `skfr_max_inner_iter`: Runs each SKFR for up to this many iterations or until convergence. Default 50. 
- `admix_n_iter`: Maximum number of iterations for ADMIXTURE. Default 1000. 
- `admix_rtol`: Convergence criteria in terms of relative change in loglikelihood. Default 1e-7. 
- `admix_n_em_iter`: Number of EM iterations to get a good initial guess for estimation. Default 5. 
- `Q`: Number of steps to be used in quasi-Newton acceleration. Default 3. 

The output are:
- `d`: the strucutre to store Admixture data. In particular, `d.p` stores the allele frequencies and `d.q` stores the admixture proportions. 
- `clusters`: cluster labels of each samples, `nothing` if `sparsity == nothing`.
- `aims`: The index of the selected SNPs in the decreasing order of importance, `nothing` if `sparsity == nothing`. 

To see the admixture proportion of each sample:

In [ ]:
d.q

Each column represent each sample, and each row represent each population.

To see the haplotype frequencies of the first alleles listed in the `.bim` file accompanying the `.bed` file:

In [ ]:
d.p

Number of columns here, `108100`, is `4 * 27025`. Again, each row represent each population. Each contiguous four-column block represent frequency of four haplotypes, adding up to 1. For example, 

In [ ]:
d.p[1, 1:4]

represents haplotype frequencies for the first pair of SNPs in the PLINK file, each representing `0|0`, `0|1`, `1|0`, and `1|1`, `0` representing "allele 1" and `1` representing "allele 2".

The following shows the final loglikelihood of the parameters.

In [ ]:
d.ll_new

The following is an example with `sparsity` defined. This example uses 10000 pairs of SNPs, i.e., 20000 SNPs. 

In [ ]:
d, clusters, aims = HaploADMIXTURE.run_admixture(filename, 379, 10000, 4; T=Float64, use_gpu=false, rng=StableRNG(7856), sparsity=10000, admix_rtol=1e-5, prefix="./EUR_subset")

The following shows the clustering result of the samples:

In [ ]:
clusters

And the following shows the SNPs selected.

In [ ]:
aims

The allele frequencies can be shown as in: 

In [ ]:
d.p

!!! The order of alleles is in the order of index (as in `sort(aim)`). This can be verified by checking the `.bim` file generated along with the newly filtered `.bed` file. 

The admixture proportions can be viewed by:

In [ ]:
d.q

In [ ]:
d.ll_new

## Multithreading

If you have multiple CPU cores available, it is recommended to launch Julia with multiple threads, for example, by using `-t` option from the terminal:
```bash
julia -t 8
```

You may also set up a multithreaded Jupyter kernel following the instruction given [here](https://github.com/JuliaLang/IJulia.jl/issues/882). 

## GPU support
GPU is enabled by setting the keyword argument `use_gpu` to `true`. The parts computing gradients and Hessians of the loglikelihood is moved to GPU.